In [ ]:
#| default_exp explain

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2
from ipynb_path import *

In [ ]:
#| export
from __future__ import annotations
from relax.import_essentials import *
from relax.data_module import DataModule, load_data
from relax.base import *
from relax.methods import *
from relax.strategy import *
from relax.ml_model import *

Using JAX backend.


No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [ ]:
#| export
@dataclass
class Explanation:
    """Generated CF Explanations class."""
    cf_name: str  # cf method's name
    data_module: DataModule  # data module
    cfs: jnp.DeviceArray  # generated cf explanation of `X`
    total_time: float  # total runtime
    pred_fn: Callable[[jnp.DeviceArray], jnp.DeviceArray]  # predict function
    dataset_name: str = str()  # dataset name
    xs: jnp.ndarray = None  # input
    ys: jnp.ndarray = None  # label

    def __post_init__(self):
        if self.data_module:
            if self.dataset_name == str():
                self.dataset_name = self.data_module.name
            if self.xs is None:
                self.xs = self.data_module.xs
            if self.ys is None:
                self.ys = self.data_module.ys


In [ ]:
#| export
def prepare_pred_fn(
    cf_module: CFModule,
    data: DataModule,
    pred_fn: Callable[[Array, ...], Array], # Predictive function. 
    pred_fn_args: Dict = None,
) -> Callable[[Array], Array]: # Return predictive function with signature `(x: Array) -> Array`.
    """Prepare the predictive function for the CF module. 
    We will train the model if `pred_fn` is not provided and `cf_module` does not have `pred_fn`.
    If `pred_fn` is found in `cf_module`, we will use it irrespective of `pred_fn` argument.
    If `pred_fn` is provided, we will use it.
    """
    # Train the model if `pred_fn` is not provided.
    if not hasattr(cf_module, 'pred_fn') and pred_fn is None:
        model = MLModule().train(data)
        return model.pred_fn
    # If `pred_fn` is detected in cf_module, 
    # use it irrespective of `pred_fn` argument.
    elif hasattr(cf_module, 'pred_fn'):
        return cf_module.pred_fn
    # If `pred_fn` is provided, use it.
    else:
        if pred_fn_args is not None:
            pred_fn = ft.partial(pred_fn, **pred_fn_args)
        return pred_fn

def prepare_cf_module(
    cf_module: CFModule,
    data_module: DataModule,
    pred_fn: Callable[[Array], Array] = None,
    train_config: Dict[str, Any] = None, 
):
    """Prepare the CF module. 
    It will hook up the data module's apply functions via the `init_apply_fns` method
    (e.g., `apply_constraints_fn` and `apply_regularization_fn`).
    It will also train the model if `cf_module` is a `ParametricCFModule`.
    """
    cf_module.init_apply_fns(
        apply_constraints_fn=data_module.apply_constraints,
        apply_regularization_fn=data_module.apply_regularization,
    )
    if isinstance(cf_module, ParametricCFModule):
        cf_module.train(data_module, pred_fn=pred_fn, **train_config)
    return cf_module


In [ ]:
#| export
def generate_cf_explanations(
    cf_module: CFModule, # CF Explanation Module
    data: DataModule, # Data Module
    pred_fn: Callable[[Array, ...], Array] = None, # Predictive function
    strategy: str | BaseStrategy = None, # Parallelism Strategy for generating CFs. Default to `vmap`.
    train_config: Dict[str, Any] = None, 
    pred_fn_args: dict = None # auxiliary arguments for `pred_fn` 
) -> Explanation: # Return counterfactual explanations.
    """Generate CF explanations."""

    # Prepare `pred_fn`, `cf_module`, and `strategy`.
    pred_fn = prepare_pred_fn(cf_module, data, pred_fn, pred_fn_args)
    cf_module = prepare_cf_module(cf_module, data, train_config)
    if strategy is None:
        strategy = StrategyFactory.get_default_strategy()
    strategy = StrategyFactory.get_strategy(strategy)
    
    # Generate CF explanations.
    start_time = time.time()
    cfs = strategy(cf_module.generate_cf, data.xs, pred_fn).block_until_ready()
    total_time = time.time() - start_time

    # Return CF explanations.
    return Explanation(
        cf_name=cf_module.name,
        data_module=data,
        cfs=cfs,
        total_time=total_time,
        pred_fn=pred_fn,
    )

In [ ]:
dm = load_data("adult")
ml_model = MLModule().train(dm)

Epoch 1/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6138 - loss: 0.6874    
Epoch 2/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 0s 945us/step - accuracy: 0.8169 - loss: 0.4086
Epoch 3/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 0s 961us/step - accuracy: 0.8189 - loss: 0.3899
Epoch 4/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 0s 959us/step - accuracy: 0.8215 - loss: 0.3870
Epoch 5/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 0s 963us/step - accuracy: 0.8243 - loss: 0.3780
Epoch 6/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 0s 954us/step - accuracy: 0.8240 - loss: 0.3771
Epoch 7/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 0s 956us/step - accuracy: 0.8236 - loss: 0.3763
Epoch 8/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 0s 963us/step - accuracy: 0.8219 - loss: 0.3770
Epoch 9/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 0s 965us/step - accuracy: 0.8197 - loss: 0.3781
Epoch 10/10
191/191 ━━━━━━━━━━━━━━━━━━━━ 0s 955us/step - accuracy: 0.8274 - loss: 0.3724


In [ ]:
exps = generate_cf_explanations(
    VanillaCF(),
    dm, ml_model.pred_fn,
)

  0%|          | 0/100 [00:00<?, ?it/s]